In [88]:
import pandas
import pandas as pd
import tensorflow
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification
import numpy as np
import pandas as pd

In [89]:
dataset_train = load_dataset('csv', data_files='feedback-prize-effectiveness/train.csv')

dataset_train['labels'] = pd.Series(dataset_train['train']['discourse_effectiveness']).map({'Ineffective': 1, 'Adequate': 2, 'Effective':3}).tolist()


Using custom data configuration default-541aa7b1f33b3777
Reusing dataset csv (C:\Users\joshz\.cache\huggingface\datasets\csv\default-541aa7b1f33b3777\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

In [90]:

dataset = dataset_train['train'].train_test_split(train_size=0.8)

In [91]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use 

In [92]:
def tokenize_function(example):
    return tokenizer(example["discourse_text"],  truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'input_ids', 'attention_mask'],
        num_rows: 29412
    })
    test: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'input_ids', 'attention_mask'],
        num_rows: 7353
    })
})

In [99]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [100]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=['labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [101]:

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [102]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, {})>

In [103]:

model.fit(
    tf_train_dataset,
)

TypeError: in user code:

    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\transformers\modeling_tf_utils.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 537, in minimize
        grads_and_vars = self._compute_gradients(
    File "C:\Users